In [ ]:
import numpy as np
from itertools import permutations
from numpy.linalg import inv, cholesky, LinAlgError
import SCM_data
import MEC
import synthetic_dataset

def l0_norm(U, threshold=0.05):
    """
    Count the number of non-zero off-diagonal elements in the upper triangular matrix U.
    """
    p = U.shape[0]
    count = 0
    for i in range(p):
        for j in range(p):
            if abs(U[i, j]) > threshold:
                count += 1
    return count

def estimate_structure_from_data(X, verbose=False):
    """
    Estimate the sparsest DAG structure by searching over permutations P
    and constructing U = I - Omega^{1/2} L based on Cholesky decomposition.

    Parameters:
        X (np.ndarray): Input data matrix (n x p), where rows are samples.
        verbose (bool): If True, print debug info.

    Returns:
        best_W (np.ndarray): Estimated upper triangular matrix (structure).
        best_Omega (np.ndarray): Diagonal noise precision matrix.
        best_P (np.ndarray): Best permutation matrix.
    """
    n, p = X.shape
    Sigma_hat = np.cov(X, rowvar=False)  # Step 1

    best_score = np.inf
    best_W, best_Omega, best_P = None, None, None

    for perm in permutations(range(p)):
        P = np.eye(p)[list(perm)]  # permutation matrix
        #print("Try permutation:")
        #print(P)
        # Step 2: permute covariance
        Sigma_perm = P @ Sigma_hat @ P.T

        try:
            # Step 3: compute precision matrix
            Theta = inv(Sigma_perm)
            # print("Theta =\n",Theta)

            # Step 4: Cholesky decomposition (Theta = L L^T)
            L = cholesky(Theta)

            # Step 5: compute Omega and W = I - Omega^{1/2} L
            diag_L = np.diag(L)
            # print("diag_L =\n",diag_L)
            Omega = np.diag(1.0 / (diag_L**2))
            #Omega = diag_L.T**2
            sqrt_Omega = np.diag(1.0 / diag_L)
            #sqrt_Omega = diag_L.T
            W = np.eye(p) -  L @ sqrt_Omega
            # print(W)

            score = l0_norm(W)
            # print("score =",score)
            if score < best_score:
                best_score = score
                best_W = W
                best_Omega = Omega
                best_P = P
                best_L = L
                if verbose:
                    print(f"New best score {score} with permutation {perm}")
        except LinAlgError:
            continue

    return best_W, best_Omega, best_P, best_L, best_score
    


In [6]:
def weight_to_adjacency(W, threshold=0.05):
    """
    Convert a weight matrix to an adjacency matrix.
    
    Parameters:
        W (np.ndarray): Weight matrix (square matrix).
        threshold (float): Values with absolute weight <= threshold are treated as 0.
    
    Returns:
        np.ndarray: Binary adjacency matrix of the same shape.
    """
    if not isinstance(W, np.ndarray):
        raise TypeError("Input W must be a numpy array.")
    if W.shape[0] != W.shape[1]:
        raise ValueError("Input W must be a square matrix.")
    
    G = (np.abs(W) > threshold).astype(int)
    return G

In [7]:
times = 100
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = SCM_data.generate_scm_data(i,10000, seed = seed)
        data = np.array([X, Y, Z]).T
        #print(data.T@ data / 10000)
        W_est, Omega_est, P_est, L_est, score = estimate_structure_from_data(data,verbose=False)
        W_est = P_est.T @ W_est @ P_est
        G_est = weight_to_adjacency(W_est, 0.05)
        #print(Omega_est)
        #print(inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
        #print("G_true = \n",G_true)
        #print("G_est = \n",G_est)
        if MEC.is_in_markov_equiv_class(G_true, G_est): true_count[i-1] += 1
    print(f"SCM {i} : {true_count[i-1]/times}")


SCM 1 : 1.0
SCM 2 : 1.0
SCM 3 : 1.0
SCM 4 : 1.0
SCM 5 : 1.0


In [4]:
X, Y, Z, G_true, CPDAG = SCM_data.generate_scm_data(5,10000)
data = np.array([X, Y, Z]).T
true_cov = np.cov(data.T)
print("true_cov = \n",true_cov)
true_precision = np.linalg.inv(true_cov)
#print("true_precision = \n",true_precision)
W_est, Omega_est, P_est, L_est, score = estimate_structure_from_data(data,verbose=True)
W_est = P_est.T @ W_est @ P_est
#print("L_est = \n",L_est)
#print("L_est @ L_est.T = \n",L_est @ L_est.T)
print("W_est= \n",W_est)
#print("Omega_est = \n",Omega_est)
#print("score = \n",score)
#print("P_est = \n",P_est)
print("Estimated cov = \n",inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
G_est = weight_to_adjacency(W_est)
print("G_true = \n",G_true)
print("G_est = \n",G_est)

true_cov = 
 [[ 1.00598488  1.00731324  5.02628322]
 [ 1.00731324  2.00837269  8.02818927]
 [ 5.02628322  8.02818927 35.08236363]]
New best score 3 with permutation (0, 1, 2)
W_est= 
 [[ 0.          0.          0.        ]
 [-0.83458065  0.          0.        ]
 [ 0.33425498  0.22883832  0.        ]]
Estimated cov = 
 [[ 1.00598488  1.00731324  5.02628322]
 [ 1.00731324  2.00837269  8.02818927]
 [ 5.02628322  8.02818927 35.08236363]]
G_true = 
 [[0 1 1]
 [0 0 1]
 [0 0 0]]
G_est = 
 [[0 0 0]
 [1 0 0]
 [1 1 0]]


In [5]:
G_true = np.array([[0,0,0,1],[0,0,0,1],[0,0,0,0],[0,0,0,0]])
X = synthetic_dataset.dataset_based_on_B(G_true,1000,1,'gaussian_ev')
W_est, Omega_est, P_est, L_est, score = estimate_structure_from_data(X,verbose=True)
W_est = P_est.T @ W_est @ P_est
G_est = weight_to_adjacency(W_est)
print(G_est)
print(G_true)

New best score 4 with permutation (0, 1, 2, 3)
New best score 3 with permutation (1, 0, 2, 3)
New best score 2 with permutation (3, 2, 0, 1)
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]
